In [1]:
import cantera as ct 
from rmgpy import chemkin
from rmgpy import kinetics

In [10]:
def compare_rxn_equations(modelR_reactions, modelN_reactions):
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (modelR_reactions, modelN_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    for reactionList, reaction in to_remove:
        reactionList.remove(reaction)
    print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = []
    common_reactions_equations = []
    unique_reactionsR = []
    unique_reactionsN = []
    copy_of_modelN_reactions = modelN_reactions.copy() #list 1
    copy_of_modelR_reactions = modelR_reactions.copy() #list 2

    
    for rxnN in copy_of_modelN_reactions:
        for rxnR in copy_of_modelR_reactions:  # make a copy so you don't remove from the list you are iterating over
            if rxnN.is_isomorphic(rxnR):
                common_reactions.append([rxnN, rxnR])
                common_reactions_equations.append([str(rxnN), str(rxnR)])
                # Remove reaction 2 from being chosen a second time.
                # Let each reaction only appear only once in the diff comparison.
                # Otherwise this miscounts number of reactions in model 2.
                copy_of_modelR_reactions.remove(rxnR)
 
    
    for rxnN in copy_of_modelN_reactions:
        for rN, rR in common_reactions:
            if rxnN is rN:
                break
        else:
            unique_reactionsN.append(rxnN)
    for rxnR in copy_of_modelR_reactions:
        for rN, rR in common_reactions:
            if rxnR is rR:
                break
        else:
            unique_reactionsR.append(rxnR)
            
    different_kinetics_equations = []
    different_kinetics_reactions = []
            
    for rN, rR in common_reactions: 
        if (rN.kinetics.is_identical_to(rR.kinetics)==False) and (rN.kinetics.is_similar_to(rR.kinetics)==False):
            different_kinetics_equations.append(str(rR))
            different_kinetics_reactions.append(rR)
            
            
        
    print(len(different_kinetics_equations))
    print(different_kinetics_equations)
    return different_kinetics_equations, different_kinetics_reactions
    

def execute(full_pathRMG, chemkinRMG, dictionaryRMG, full_pathNIST, chemkinNIST, dictionaryNIST): 
    '''
    loads in chemkin files and runs commands to execute the file
    '''
    full_pathN = full_pathNIST
    inp_pathN = full_pathNIST + chemkinNIST
    dict_pathN = full_pathNIST + dictionaryNIST
    modelN_species, modelN_reactions = chemkin.load_chemkin_file(inp_pathN, dict_pathN)

    full_pathR = full_pathRMG
    inp_pathR = full_pathRMG + chemkinRMG
    dict_pathR = full_pathRMG + dictionaryRMG
    modelR_species, modelR_reactions = chemkin.load_chemkin_file(inp_pathR, dict_pathR)
    
    compare_rxn_equations(modelR_reactions, modelN_reactions)
    


execute(
    "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/",
    "no_halogens.inp",
    "no_halogens_dict.txt",
    "/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/",
    "pym_gas_copy.inp",
    "species_dictionary_2-BTP.txt"
    )


    

Following reactions removed because they have unknown species: []
97
['H2(8) <=> H(5) + H(5)', 'H2O(9) <=> H(5) + OH(7)', 'H(5) + HO2(12) <=> O2(4) + H2(8)', 'H2O2(13) <=> OH(7) + OH(7)', 'OH(7) + HO2(12) <=> O2(4) + H2O(9)', 'OH(7) + H2O2(13) <=> HO2(12) + H2O(9)', 'O(6) + CO(14) <=> CO2(15)', 'OH(7) + HCO(16) <=> H2O(9) + CO(14)', 'HCO(16) <=> H(5) + CO(14)', 'CH2O(21) <=> H2(8) + CO(14)', 'H2(8) + CH(18) <=> H(5) + CH2(T)(19)', 'O2(4) + CH(18) <=> O(6) + HCO(16)', 'H(5) + HCO(16) <=> CH2O(21)', 'H(5) + CH2(T)(19) <=> CH3(20)', 'OH(7) + CH2(T)(19) <=> H(5) + CH2O(21)', 'OH(7) + CH2(T)(19) <=> H2O(9) + CH(18)', 'H2O(9) + CH2(S)(26) <=> CH3OH(27)', 'CH2OH(34) <=> H(5) + CH2O(21)', 'CH3O(28) <=> H(5) + CH2O(21)', 'H(5) + CH3(20) <=> CH4(3)', 'OH(7) + CH3(20) <=> CH3OH(27)', 'OH(7) + CH3(20) <=> H2O(9) + CH2(S)(26)', 'O2(4) + CH3(20) <=> O(6) + CH3O(28)', 'O2(4) + CH3(20) <=> OH(7) + CH2O(21)', 'HO2(12) + CH3(20) <=> O2(4) + CH4(3)', 'HO2(12) + CH4(3) <=> H2O2(13) + CH3(20)', 'H(5) + CH3